A notebook to calculate comparison metrics for the GAN reconstructions, such as the Pearson correlation coefficient and RMSE.

In [1]:
# Imports list
import numpy as np
import matplotlib.pyplot as plt
from tabulate import tabulate
import yaml
import json
import types
import sys
dir = '/home/jjwhit/rcGAN/'
sys.path.append(dir)
from mass_map_utils.scripts.ks_utils import rmse, pearsoncoeff, psnr, snr
from data.lightning.MassMappingDataModule import MMDataTransform
from mass_map_utils.scripts.ks_utils import backward_model
from scipy import ndimage


In [ ]:
def load_object(dct):
    return types.SimpleNamespace(**dct)

In [ ]:
with open(dir+'configs/mass_map.yml', 'r') as f:
    cfg = yaml.load(f, Loader=yaml.FullLoader)
    cfg = json.loads(json.dumps(cfg), object_hook=load_object)


In [32]:
# Load the samples (there are 20 sets of data):

data_dir = cfg.save_path

np_gts = {f"{i}": np.load(data_dir+f"np_gt_{i}.npy") for i in range(1, 11)}
np_samps = {f"{i}": np.load(data_dir+f"np_samps_{i}.npy") for i in range(1, 11)}
np_avgs = {f"{i}": np.load(data_dir+f"np_avgs_{i}.npy") for i in range(1, 11)}
np_stds = {f"{i}": np.load(data_dir+f"np_stds_{i}.npy") for i in range(1, 11)}


#array['i'] to access the i-th ground truth, reconstruction, standard deviation, and Kaiser-Squires map; 
# for individual posterior samples np_samps['i']['j'] where j is in range [0,31].

mask =  np.load(
    data_dir + 'cosmos_mask.npy', allow_pickle=True
).astype(bool)
std1 = np.load(data_dir + 'cosmos_std1.npy', allow_pickle=True)
std2 = np.load(data_dir + 'cosmos_std2.npy', allow_pickle=True)

In [ ]:
kernel = MMDataTransform.compute_fourier_kernel(cfg.im_size)
np_kss = {}
for i in range(1, 11):
    num = f'{i}'
    gamma_sim = MMDataTransform.forward_model(np_gts[num], kernel) + (
                std1 * np.random.randn(cfg.im_size, cfg.im_size) + 1.j * std2 * np.random.randn(cfg.im_size,cfg.im_size)
            )
    gamma_sim *= mask
    backward = backward_model(gamma_sim, kernel)
    ks = ndimage.gaussian_filter(backward, sigma=1/.29)

    np.save(data_dir+f'np_kss_{i}.npy', ks)
    np_kss[num] = ks

In [ ]:
# np_kss = {f"{i}": np.load(data_dir+f"np_kss_{i}.npy") for i in range(1, 11)}

In [37]:
# Calculate the RMSE and Pearson correlation coefficient for the Kaiser-Squires and GAN reconstructions for the 20 samples

r_ks = []
r_gan = []
rmse_ks = []
rmse_gan = []
psnr_ks = []
psnr_gan = []
snr_ks = []
snr_gan = []

for n in range(1,11):
    num = f'{n}'
    # Gets the real component of the truth, Kaiser-Squires, and GAN reconstruction
    gt = np_gts[num].real
    ks = np_kss[num].real
    gan = np_avgs[num].real

    r_gan.append(pearsoncoeff(gt, gan, mask))
    r_ks.append(pearsoncoeff(gt, ks, mask))

    rmse_ks.append(rmse(ks, gt, mask))
    rmse_gan.append(rmse(gan, gt, mask))

    psnr_ks.append(psnr(gt, ks, mask))
    psnr_gan.append(psnr(gt, gan, mask))

    snr_ks.append(snr(gt, ks, mask))
    snr_gan.append(snr(gt, gan, mask))


In [39]:
head = ['Metric', 'Kaiser-Squires', 'GAN']
table = [['Pearson (high)', np.mean(r_ks), np.mean(r_gan)],['RMSE (low)', np.mean(rmse_ks), np.mean(rmse_gan)],['Psnr (high)', np.mean(psnr_ks), np.mean(psnr_gan)],['snr (high)', np.mean(snr_ks), np.mean(snr_gan)]]
print('psnr')
print(tabulate(table, headers=head, tablefmt='pretty'))

psnr
+----------------+----------------------+----------------------+
|     Metric     |    Kaiser-Squires    |         GAN          |
+----------------+----------------------+----------------------+
| Pearson (high) |  0.620139486557506   |  0.716211199760437   |
|   RMSE (low)   | 0.022737655433688005 | 0.020701179280877113 |
|  PSNR (high)   |  30.185091395580883  |  31.012511253356934  |
|   SNR (high)   |  2.0382314360463196  |  2.8656497299671173  |
+----------------+----------------------+----------------------+
